# RPG dice

In roleplaying games, conflicts between players are often decided by rolling
dice, often multiple dice at the same time. Furthermore games often use
not only the well known 6-sided dice, but also 4-sided, 8-sided, 20-sided dice, etc.

The number and type of dice is described by the following notation:

```text
<number of dice> d <number of sides per die>
```

For example, rolling two 6-sided dice is described as `2d6`.
Sometimes more complex formulas are used: `3d20 + 2d6 - 4`
means that three 20-sided dice and two 6-sided dice are rolled
at the same time, and the total sum of numbers is then reduced by 4.

In some games, rolling the lowest or highest number of dice is treated
in a special way ("catastrophic failure", "critical success").

In the following exercise your task is to implement RPG dice in Python.
To simplify testing your implementation it might be advisable
to implement it in an IDE, but it is also possible to write tests as
assertions in a jupyter notebook. 

Write tests for each functionality you implement. How can you deal with the
randomness in dice rolling? What are the strengths and weaknesses of the strategy
you have chosen to test?

## Generalized dice: Interface `Dice`

Implement an abstract class `Dice` that describes rolling of
arbitrary dice: The class should provide the following abstract methods:

- `roll(): int` returns the result of a roll with the appropriate
  dice.

- properties `max_value: int` and `min_value: int` return the smallest and largest
  value that can be rolled with the corresponding dice.
  dice.

If you are using Pytest:
Write parametric Pytest tests to test (indirect) instances of `Dice`.

In [ ]:
import random
from abc import ABC, abstractmethod
from typing import Tuple, Callable, Sequence, Union, Iterable

In [ ]:
class Dice(ABC):
    """Roll with a combination of multiple dice."""

    @abstractmethod
    def roll(self) -> int:
        ...

    @property
    @abstractmethod
    def min_value(self) -> int:
        ...

    @property
    @abstractmethod
    def max_value(self) -> int:
        ...

## Class `ConstantDice`

Implement a class `ConstantDice`, which implements the interface `Dice` and
represents a die that always rolls a constant value. The result value is
specified when the creating an instance of the class.

In [ ]:
class ConstantDice(Dice):
    def __init__(self, value):
        self.value = value

    def __eq__(self, other):
        if isinstance(other, ConstantDice):
            return self.value == other.value
        else:
            return False

    def roll(self) -> int:
        return self.value

    @property
    def min_value(self) -> int:
        return self.value

    @property
    def max_value(self) -> int:
        return self.value

In [ ]:
dice = ConstantDice(3)
assert dice.min_value == 3
assert dice.max_value == 3
assert dice.roll() == 3

## Class `FairDice`

Implement a class `FairDice` that implements the interface `Dice`and
represents a single rolls with multiple dice all having the same (configurable)
number of sides. For example, instances of the class `FairDice` should be
able to simulate dice rolls of the form `3d6` or `4d17`
(although in reality there is no 17-sided (fair) die).

How can you deal with randomness in dice rolling when testing? What
are strengths or weaknesses of the chosen testing strategy?

*Hint:* Look at `random.randint()` and `random.seed()`.

In [ ]:
class FairDice(Dice):
    def __init__(self, num_dice, num_sides):
        assert num_dice >= 1
        assert num_sides >= 2

        self.num_dice = num_dice
        self.num_sides = num_sides

    def __eq__(self, other):
        if isinstance(other, FairDice):
            return (self.num_dice == other.num_dice
                    and self.num_sides == other.num_sides)
        else:
            return False

    def roll(self) -> int:
        result = 0
        for _ in range(self.num_dice):
            result += random.randint(1, self.num_sides)
        return result

    @property
    def min_value(self) -> int:
        return self.num_dice

    @property
    def max_value(self) -> int:
        return self.num_sides * self.num_dice

In [ ]:
dice = FairDice(2, 6)
assert dice.min_value == 2
assert dice.max_value == 12
random.seed(1)
assert dice.roll() == 7

## Class `SumDice`

Implement a class `SumDice` which implements the interface `Dice` and
represents the sum of the dice rolls with several different dice
(potentially of the form `<m>d<n>`).

In [ ]:
class SumDice(Dice):
    def __init__(self, dice: Iterable):
        assert dice
        self.dice = dice

    def __eq__(self, other):
        if isinstance(other, SumDice):
            return self.dice == other.dice
        else:
            return False

    def roll(self) -> int:
        return sum(d.roll() for d in self.dice)

    @property
    def min_value(self) -> int:
        return sum(d.min_value for d in self.dice)

    @property
    def max_value(self) -> int:
        return sum(d.max_value for d in self.dice)



In [ ]:
dice = SumDice([ConstantDice(3), ConstantDice(2)])
assert dice.min_value == 5
assert dice.max_value == 5
assert dice.roll() == 5

In [ ]:
dice = SumDice([ConstantDice(1), FairDice(2, 6)])
assert dice.min_value == 3
assert dice.max_value == 13
random.seed(1)
assert dice.roll() == 8

## Class `SimpleDie`

Implement a class `SimpleDie`, which implements the interface `Dice`
and represents a single roll of a die with an arbitrary number of sides
(specified when the die is instantiated).

In [ ]:
class SimpleDie(Dice):
    def __init__(self, num_sides):
        assert num_sides >= 2
        self.num_sides = num_sides

    def __eq__(self, other):
        if isinstance(other, SimpleDie):
            return self.num_sides == other.num_sides
        else:
            return False

    def roll(self) -> int:
        return random.randint(1, self.num_sides)

    @property
    def min_value(self) -> int:
        return 1

    @property
    def max_value(self) -> int:
        return self.num_sides

In [ ]:
die = SimpleDie(6)
assert die.min_value == 1
assert die.max_value == 6
random.seed(1)
assert die.roll() == 2

## Class `MultipleRollDice`

Implement a class `MultipleRollDice` which implements the interface `Dice` 
and represents rolling of a (generalized) dice a certain number of times.
Both the die and the number of rolls should be specified when an
instance is created.

What is the relationship between `FairDice` and the combination of
`SimpleDie` and `MultipleRollDice`? How do the two
implementation strategies in their testability.

In [ ]:
class MultipleRollDice(Dice):
    def __init__(self, rolls, dice):
        assert rolls >= 1
        assert dice
        self.rolls = rolls
        self.dice = dice

    def __eq__(self, other):
        if isinstance(other, MultipleRollDice):
            return self.rolls == other.rolls and self.dice == other.dice
        else:
            return False

    def roll(self) -> int:
        result = 0
        for _ in range(self.rolls):
            result += self.dice.roll()
        return result

    @property
    def min_value(self) -> int:
        return self.rolls * self.dice.min_value

    @property
    def max_value(self) -> int:
        return self.rolls * self.dice.max_value



In [ ]:
dice = MultipleRollDice(2, SimpleDie(6))
assert dice.min_value == 2
assert dice.max_value == 12
random.seed(1)
assert dice.roll() == 7

# Factory for RPG dice

Write a function `create_dice(configuration: str) -> Dice`,
which gets a configuration of RPG dice as argument and returns an
suitable configuration of dice instances.  For example
for "3d8 + 6" a `SumDice` should be returned, which contains a `FairDice` (with
3 8-sided dice) and a `ConstantDice` with the value 6.